***
# Model Deployment : Detecting and Analyzing Machine Learning Model Drift Using Open-Source Monitoring Tools

***
### [**John Pauline Pineda**](https://github.com/JohnPaulinePineda) <br> <br> *September 30, 2025*
***

* [**1. Table of Contents**](#TOC)
    * [1.1 Data Background](#1.1)
    * [1.2 Data Description](#1.2)
    * [1.3 Data Quality Assessment](#1.3)
    * [1.4 Data Preprocessing](#1.4)
        * [1.4.1 Data Splitting](#1.4.1)
        * [1.4.2 Outlier and Distributional Shape Analysis](#1.4.4)
        * [1.4.3 Collinearity](#1.4.5)
    * [1.5 Data Exploration](#1.5)
        * [1.5.1 Exploratory Data Analysis](#1.5.1)
        * [1.5.2 Hypothesis Testing](#1.5.2)
    * [1.6 Premodelling Data Preparation](#1.6)
        * [1.6.1 Preprocessed Data Description](#1.6.1)
        * [1.6.2 Preprocessing Pipeline Development](#1.6.2)
    * [1.7 Model Development and Validation](#1.7)
        * [1.7.1 Random Forest](#1.7.1)
        * [1.7.2 AdaBoost](#1.7.2)
        * [1.7.3 Gradient Boosting](#1.7.3)
        * [1.7.4 XGBoost](#1.7.4)
        * [1.7.5 Light GBM](#1.7.5)
        * [1.7.6 CatBoost](#1.7.6)
    * [1.8 Model Monitoring using the NannyML Framework](#1.8)
        * [1.8.1 Baseline Control](#1.8.1)
        * [1.8.2 Simulated Covariate Drift](#1.8.2)
        * [1.8.3 Simulated Prior Shift](#1.8.3)
        * [1.8.4 Simulated Concept Drift](#1.8.4)
        * [1.8.5 Simulated Missingness Spike](#1.8.5)
        * [1.8.6 Simulated Seasonal Pattern](#1.8.6)
* [**2. Summary**](#Summary)   
* [**3. References**](#References)

***

# 1. Table of Contents <a class="anchor" id="TOC"></a>

This project investigates **open-source frameworks for post-deployment model monitoring and performance estimation**, with a particular focus on **NannyML** or detecting and interpreting shifts in machine learning pipelines using <mark style="background-color: #CCECFF"><b>Python</b></mark>. The objective was to systematically analyze how different types of drift and distribution changes manifest after model deployment, and to demonstrate how robust monitoring mitigates risks of performance degradation and biased decision-making. The workflow began with the development and selection of a baseline predictive model, which serves as a reference for stability. The dataset was then deliberately perturbed to simulate a range of realistic post-deployment scenarios: **Covariate Drift** (shifts in feature distributions), **Prior Shift** (changes in target label proportions), **Concept Drift** (evolving relationships between features and outcomes), **Missingness Spikes** (abrupt increases in absent data), and **Seasonal Patterns** (periodic variations in distributions). NannyML’s statistical tests, visualization capabilities, and performance estimation methods were subsequently applied to diagnose these shifts, evaluate their potential impact, and provide interpretable insights into model reliability. By contrasting baseline and perturbed conditions, the experiment demonstrated how continuous monitoring augments traditional offline evaluation, offering a safeguard against hidden risks. The findings highlighted how tools like NannyML can integrate seamlessly into MLOps workflows to enable proactive governance, early warning systems, and sustainable deployment practices. All results were consolidated in a [<span style="color: #FF0000"><b>Summary</b></span>](#Summary) presented at the end of the document.

[Post-Deployment Monitoring](https://www.nannyml.com/) refers to the continuous oversight of machine learning models once they are integrated into production systems. Unlike offline evaluation, which relies on static validation datasets, monitoring addresses the challenges of evolving real-world data streams where underlying distributions may shift. Effective monitoring ensures that models remain accurate, unbiased, and aligned with business objectives. In MLOps, monitoring encompasses data integrity checks, drift detection, performance estimation, and alerting mechanisms. NannyML operationalizes this concept by focusing on performance estimation without ground truth, and by offering statistical methods to detect when data or predictions deviate from expected baselines. The challenges of post-deployment monitoring include delayed or missing ground truth labels, non-stationary data, hidden feedback loops, and difficulties distinguishing natural fluctuations from problematic drifts. Common solutions involve deploying drift detection algorithms, conducting regular audits of data pipelines, simulating counterfactuals, and retraining models on updated data. Monitoring frameworks must balance sensitivity (detecting real problems quickly) with robustness (avoiding false alarms caused by natural noise). Another key challenge is explainability: stakeholders need interpretable signals that justify interventions such as retraining or rolling back models. Tools like NannyML address these challenges through statistical tests for data drift, performance estimation without labels, missingness tracking, and visual diagnostics, making monitoring actionable for data scientists and business teams alike.

[Covariate Shift](https://www.nannyml.com/) occurs when the distribution of input features changes over time compared to the data used to train the model. Also known as data drift, it does not necessarily imply that the model’s predictive mapping is invalid, but it often precedes performance degradation. Detecting covariate drift requires comparing feature distributions between baseline (reference) data and incoming production data. NannyML provides multiple statistical tests and visualization tools to flag significant changes. Key signatures of covariate shift include shifts in summary statistics (mean, variance), changes in distributional shape, or increased divergence between reference and production feature distributions. These shifts may lead to poor generalization, as the model has not been exposed to the altered feature ranges. Detection techniques include univariate statistical tests (e.g., Kolmogorov–Smirnov, Chi-square), multivariate distance measures (e.g., Jensen–Shannon divergence, Population Stability Index), and density estimation methods. Remediation approaches involve domain adaptation, re-weighting training samples, or retraining models on updated data distributions. NannyML implements univariate and multivariate tests, provides drift magnitude quantification, and visualizes feature-level changes, allowing practitioners to pinpoint which features are most responsible for the detected drift.

[Prior Shift](https://www.nannyml.com/) arises when the distribution of the target variable changes, while the conditional relationship between features and labels remains stable. This is also referred to as label shift. Models trained on the original distribution may underperform because their predictions no longer match the new class priors. Detecting prior shifts is crucial, especially in imbalanced classification tasks where small changes in priors can lead to large performance impacts. Prior shift is typically characterized by systematic increases or decreases in class frequencies without corresponding changes in feature distributions. Its impact includes skewed decision thresholds, inflated false positives/negatives, and degraded calibration of predicted probabilities. Detection approaches include monitoring predicted class proportions, estimating priors using EM-based algorithms, and re-weighting predictions to align with new distributions. Correction strategies may involve resampling, threshold adjustment, or cost-sensitive learning. NannyML assists by tracking predicted probability distributions and comparing them against reference priors, using techniques such as KL divergence and PSI to quantify the magnitude of shift.

[Concept Drift](https://www.nannyml.com/) occurs when the underlying relationship between input features and target labels evolves over time. Unlike covariate shift, where features change independently, concept drift implies that the model’s mapping function itself becomes outdated. Concept drift is among the most damaging forms of drift because it directly undermines predictive accuracy. Detecting it often requires monitoring model outputs or inferred performance over time. NannyML addresses this by estimating performance even when ground truth labels are unavailable. Concept drift is typically signaled by a gradual or sudden decline in performance metrics, inconsistent error patterns, or misalignment between expected and actual prediction behavior. Its impact is severe: models may lose predictive power entirely if they cannot adapt. Detection methods include window-based performance monitoring, hypothesis testing, adaptive ensembles, and statistical monitoring of residuals. Corrective actions include periodic retraining, incremental learning, and online adaptation strategies. NannyML leverages Confidence-Based Performance Estimation (CBPE) and other statistical techniques to estimate performance degradation without labels, making it possible to detect concept drift in real-time production environments.

[Missingness Spike](https://www.nannyml.com/) refers to sudden increases in missing values within production data. Missing features can destabilize preprocessing pipelines, distort predictions, and signal upstream data collection failures. Monitoring missingness is critical for ensuring both model reliability and data pipeline health. NannyML provides built-in mechanisms to track and visualize changes in missing data patterns, alerting stakeholders before downstream impacts occur. Key indicators of missingness spikes include abrupt rises in null counts, missing categorical levels, or structural breaks in feature completeness. The consequences range from biased predictions to outright system failures if preprocessing pipelines cannot handle unexpected missingness. Detection methods include statistical monitoring of missing value proportions, anomaly detection on completeness metrics, and threshold-based alerts. Solutions typically involve robust imputation, pipeline hardening, and upstream data validation. NannyML offers automated missingness detection, completeness trend visualization, and configurable thresholds, ensuring that missingness issues are surfaced early.

[Seasonal Pattern Shift](https://www.nannyml.com/) represents periodic fluctuations in data distributions or outcomes that follow predictable cycles. If models are not trained with sufficient historical data to capture these patterns, their predictions may systematically underperform during certain periods. NannyML’s monitoring can reveal recurring deviations, helping teams distinguish between natural seasonality and genuine drift that requires retraining. Seasonality is often characterized by cyclic patterns in data features, prediction distributions, or performance metrics. Its impact includes systematic biases, recurring error peaks, and difficulty distinguishing drift from natural variability. Detection techniques include autocorrelation analysis, Fourier decomposition, and seasonal-trend decomposition. Mitigation strategies involve training with longer historical datasets, adding time-related features, or developing seasonally adaptive models. NannyML highlights recurring deviations in drift metrics, making it easier for practitioners to separate cyclical behavior from true degradation, ensuring that alerts are contextually relevant.

[Performance Estimation Without Labels](https://www.nannyml.com/) refers to scenarios in real-world deployments where the ground truth often arrives with delays—or may never be available. This makes direct performance tracking difficult. NannyML addresses this challenge by providing algorithms to estimate model performance without labels using confidence distributions, statistical inference, and robust estimation techniques. This capability allows practitioners to maintain visibility into model health continuously, even in label-scarce settings, bridging a critical gap in MLOps monitoring practices. Algorithms in this domain include Confidence-Based Performance Estimation (CBPE), which infers performance by comparing predicted probability distributions against expected confidence intervals, and Direct Loss Estimation, which approximates error rates based on calibration. Statistical inference techniques allow practitioners to construct confidence bounds around estimated metrics, while robust estimation mitigates the risk of spurious signals caused by small sample sizes or noisy predictions. NannyML provides implementations of CBPE and DLE, supporting metrics such as precision, recall, F1-score, and AUROC, all estimated without labels. This makes it possible to detect when a model is underperforming even before labels are collected, reducing blind spots in production monitoring.

[Performance Estimation With Labels](https://www.nannyml.com/) refers to the direct evaluation of model predictions against actual ground truth outcomes once labels are available. Unlike label-free methods, this approach allows for precise calculation of traditional performance metrics such as accuracy, precision, recall, F1-score, AUROC, and calibration error. Monitoring with labels provides the most reliable indication of model performance, enabling fine-grained diagnosis of errors and biases. The advantage of having labels is the ability to attribute errors to specific subgroups, detect fairness violations, and conduct targeted retraining. Challenges include label delay, annotation quality, and ensuring that labels accurately reflect the operational environment. Common approaches include sliding window evaluation, where performance is tracked over recent data batches, and benchmark comparison, where production metrics are compared to baseline test set results. NannyML incorporates labeled performance tracking alongside its label-free estimators, allowing users to validate estimates once ground truth becomes available. This dual capability ensures consistency, improves confidence in label-free methods, and provides a comprehensive framework for performance monitoring in both short-term and long-term horizons.

## 1.1. Data Background <a class="anchor" id="1.1"></a>

An open [Breast Cancer Dataset](https://www.kaggle.com/datasets/wasiqaliyasir/breast-cancer-dataset) from [Kaggle](https://www.kaggle.com/) (with all credits attributed to [Wasiq Ali](https://www.kaggle.com/wasiqaliyasir)) was used for the analysis as consolidated from the following primary sources: 
1. Reference Repository entitled **Differentiated Thyroid Cancer Recurrence** from [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/915/differentiated+thyroid+cancer+recurrence)
2. Research Paper entitled **Nuclear Feature Extraction for Breast Tumor Diagnosis** from the [Electronic Imaging](https://www.spiedigitallibrary.org/conference-proceedings-of-spie/1905/1/Nuclear-feature-extraction-for-breast-tumor-diagnosis/10.1117/12.148698.short)

This study hypothesized that the cell nuclei features derived from digitized images of fine needle aspirates (FNA) of breast masses influence breast cancer diagnoses between patients.

The dichotomous categorical variable for the study is:
* <span style="color: #FF0000">diagnosis</span> - Status of the patient (Malignant, Medical diagnosis of a cancerous breast tumor | Benign, Medical diagnosis of a non-cancerous breast tumor)

The predictor variables for the study are:
* <span style="color: #FF0000">radius_mean</span> - Mean of the radius measurements (Mean of distances from center to points on the perimeter)
* <span style="color: #FF0000">texture_mean</span> - Mean of the texture measurements (Standard deviation of grayscale values)
* <span style="color: #FF0000">perimeter_mean</span> - Mean of the perimeter measurements
* <span style="color: #FF0000">area_mean</span> - Mean of the area measurements
* <span style="color: #FF0000">smoothness_mean</span> - Mean of the smoothness measurements (Local variation in radius lengths)
* <span style="color: #FF0000">compactness_mean</span> - Mean of the compactness measurements (Perimeter² / area - 1.0)
* <span style="color: #FF0000">concavity_mean</span> - Mean of the concavity measurements (Severity of concave portions of the contour)
* <span style="color: #FF0000">concave points_mean</span> - Mean of the concave points measurements (Number of concave portions of the contour)
* <span style="color: #FF0000">symmetry_mean</span> - Mean of the symmetry measurements
* <span style="color: #FF0000">fractal_dimension_mean</span> - Mean of the fractal dimension measurements (Coastline approximation - 1)
* <span style="color: #FF0000">radius_se</span> - Standard error of the radius measurements (Standard error of distances from center to points on the perimeter)
* <span style="color: #FF0000">texture_se</span> - Standard error of the texture measurements (Standard deviation of grayscale values)
* <span style="color: #FF0000">perimeter_se</span> - Standard error of the perimeter measurements
* <span style="color: #FF0000">area_se</span> - Standard error of the area measurements
* <span style="color: #FF0000">smoothness_se</span> - Standard error of the smoothness measurements (Local variation in radius lengths)
* <span style="color: #FF0000">compactness_se</span> - Standard error of the compactness measurements (Perimeter² / area - 1.0)
* <span style="color: #FF0000">concavity_se</span> - Standard error of the concavity measurements (Severity of concave portions of the contour)
* <span style="color: #FF0000">concave points_se</span> - Standard error of the concave points measurements (Number of concave portions of the contour)
* <span style="color: #FF0000">symmetry_se</span> - Standard error of the symmetry measurements
* <span style="color: #FF0000">fractal_dimension_se</span> - Standard error of the fractal dimension measurements (Coastline approximation - 1)
* <span style="color: #FF0000">radius_worst</span> - Largest value of the radius measurements (Largest value of distances from center to points on the perimeter)
* <span style="color: #FF0000">texture_worst</span> - Largest value of the texture measurements (Standard deviation of grayscale values)
* <span style="color: #FF0000">perimeter_worst</span> - Largest value of the perimeter measurements
* <span style="color: #FF0000">area_worst</span> - Largest value of the area measurements
* <span style="color: #FF0000">smoothness_worst</span> - Largest value of the smoothness measurements (Local variation in radius lengths)
* <span style="color: #FF0000">compactness_worst</span> - Largest value of the compactness measurements (Perimeter² / area - 1.0)
* <span style="color: #FF0000">concavity_worst</span> - Largest value of the concavity measurements (Severity of concave portions of the contour)
* <span style="color: #FF0000">concave points_worst</span> - Largest value of the concave points measurements (Number of concave portions of the contour)
* <span style="color: #FF0000">symmetry_worst</span> - Largest value of the symmetry measurements
* <span style="color: #FF0000">fractal_dimension_worst</span> - Largest value of the fractal dimension measurements (Coastline approximation - 1)

## 1.2. Data Description <a class="anchor" id="1.2"></a>

## 1.3. Data Quality Assessment <a class="anchor" id="1.3"></a>

## 1.4. Data Preprocessing <a class="anchor" id="1.4"></a>

### 1.4.1 Data Splitting<a class="anchor" id="1.4.1"></a>

### 1.4.2 Outlier and Distributional Shape Analysis<a class="anchor" id="1.4.2"></a>

### 1.4.3 Collinearity<a class="anchor" id="1.4.3"></a>

## 1.5. Data Exploration <a class="anchor" id="1.5"></a>

### 1.5.1 Exploratory Data Analysis<a class="anchor" id="1.5.1"></a>

### 1.5.2 Hypothesis Testing<a class="anchor" id="1.5.2"></a>

## 1.6. Premodelling Data Preparation <a class="anchor" id="1.6"></a>

### 1.6.1 Preprocessed Data Description<a class="anchor" id="1.6.1"></a>

### 1.6.2 Preprocessing Pipeline Development<a class="anchor" id="1.6.2"></a>

## 1.7. Model Development and Validation <a class="anchor" id="1.7"></a>

### 1.7.1 Random Forest<a class="anchor" id="1.7.1"></a>

### 1.7.2 AdaBoost<a class="anchor" id="1.7.2"></a>

### 1.7.3 Gradient Boosting<a class="anchor" id="1.7.3"></a>

### 1.7.4 XGBoost<a class="anchor" id="1.7.4"></a>

### 1.7.5 Light GBM<a class="anchor" id="1.7.5"></a>

### 1.7.6 CatBoost<a class="anchor" id="1.7.6"></a>

## 1.8. Model Monitoring using the NannyML Framework <a class="anchor" id="1.8"></a>

### 1.8.1 Baseline Control<a class="anchor" id="1.8.1"></a>

### 1.8.2 Simulated Covariate Drift<a class="anchor" id="1.8.2"></a>

### 1.8.3 Simulated Prior Shift<a class="anchor" id="1.8.3"></a>

### 1.8.4 Simulated Concept Drift<a class="anchor" id="1.8.4"></a>

### 1.8.5 Simulated Missingness Spike<a class="anchor" id="1.8.5"></a>

### 1.8.6 Simulated Seasonal Pattern<a class="anchor" id="1.8.6"></a>

## 1.9. Consolidated Findings <a class="anchor" id="1.9"></a>

# 2. Summary <a class="anchor" id="Summary"></a>

# 3. References <a class="anchor" id="References"></a>

* **[Book]** [Reliable Machine Learning](https://www.oreilly.com/library/view/reliable-machine-learning/9781098106218/) by Cathy Chen, Niall Richard Murphy, Kranti Parisa, D. Sculley and Todd Underwood
* **[Book]** [Designing Machine Learning Systems](https://www.oreilly.com/library/view/designing-machine-learning/9781098107956/) by Chip Huyen
* **[Book]** [Machine Learning Design Patterns](https://www.oreilly.com/library/view/machine-learning-design/9781098115777/) by Valliappa Lakshmanan, Sara Robinson and Michael Munn
* **[Book]** [Machine Learning Engineering](https://www.mlebook.com/wiki/doku.php) by Andriy Burkov
* **[Book]** [Engineering MLOps](https://www.oreilly.com/library/view/engineering-mlops/9781800562882/) by Emmanuel Raj
* **[Book]** [Introducing MLOps](https://www.oreilly.com/library/view/introducing-mlops/9781492083283/) by Mark Treveil, Nicolas Omont, Clément Stenac, Kenji Lefevre, Du Phan, Joachim Zentici, Adrien Lavoillotte, Makoto Miyazaki and Lynn Heidmann
* **[Book]** [Practical MLOps](https://www.oreilly.com/library/view/practical-mlops/9781098103002/) by Noah Gift and Alfredo Deza
* **[Book]** [Data Science on AWS](https://www.oreilly.com/library/view/data-science-on/9781492079385/) by Chris Fregly and Antje Barth
* **[Book]** [Ensemble Methods for Machine Learning](https://www.manning.com/books/ensemble-methods-for-machine-learning) by Gautam Kunapuli
* **[Book]** [Applied Predictive Modeling](http://appliedpredictivemodeling.com/) by Max Kuhn and Kjell Johnson
* **[Book]** [An Introduction to Statistical Learning](https://www.statlearning.com/) by Gareth James, Daniela Witten, Trevor Hastie and Rob Tibshirani
* **[Book]** [Ensemble Methods: Foundations and Algorithms](https://www.taylorfrancis.com/books/mono/10.1201/b12207/ensemble-methods-zhi-hua-zhou) by Zhi-Hua Zhou
* **[Book]** [Effective XGBoost: Optimizing, Tuning, Understanding, and Deploying Classification Models (Treading on Python)](https://www.taylorfrancis.com/books/mono/10.1201/b12207/ensemble-methods-zhi-hua-zhou) by Matt Harrison, Edward Krueger, Alex Rook, Ronald Legere and Bojan Tunguz
* **[Python Library API]** [nannyML](https://www.nannyml.com/) by NannyML Team
* **[Python Library API]** [NumPy](https://numpy.org/doc/) by NumPy Team
* **[Python Library API]** [pandas](https://pandas.pydata.org/docs/) by Pandas Team
* **[Python Library API]** [seaborn](https://seaborn.pydata.org/) by Seaborn Team
* **[Python Library API]** [matplotlib.pyplot](https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html) by MatPlotLib Team
* **[Python Library API]** [itertools](https://docs.python.org/3/library/itertools.html) by Python Team
* **[Python Library API]** [sklearn.experimental](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.experimental) by Scikit-Learn Team
* **[Python Library API]** [sklearn.preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) by Scikit-Learn Team
* **[Python Library API]** [scipy](https://docs.scipy.org/doc/scipy/) by SciPy Team
* **[Python Library API]** [sklearn.tree](https://scikit-learn.org/stable/modules/tree.html) by Scikit-Learn Team
* **[Python Library API]** [sklearn.ensemble](https://scikit-learn.org/stable/modules/ensemble.html) by Scikit-Learn Team
* **[Python Library API]** [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) by Scikit-Learn Team
* **[Python Library API]** [xgboost](https://xgboost.readthedocs.io/en/stable/python/index.html) by XGBoost Team
* **[Python Library API]** [lightgbm](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html) by LightGBM Team
* **[Python Library API]** [catboost](https://catboost.ai/docs/en/concepts/python-reference_catboostclassifier) by CatBoost Team
* **[Python Library API]** [StatsModels](https://www.statsmodels.org/stable/index.html) by StatsModels Team
* **[Python Library API]** [SciPy](https://scipy.org/) by SciPy Team
* **[Article]** [Comprehensive Comparison of ML Model Monitoring Tools: Evidently AI, Alibi Detect, NannyML, WhyLabs, and Fiddler AI](https://medium.com/@tanish.kandivlikar1412/comprehensive-comparison-of-ml-model-monitoring-tools-evidently-ai-alibi-detect-nannyml-a016d7dd8219) by Tanish Kandivlikar (Medium)
* **[Article]** [Monitoring AI in Production: Introduction to NannyML](https://adnankarol.medium.com/monitoring-ai-in-production-75f1260260cb) by Adnan Karol (Medium)
* **[Article]** [Data Drift Explainability: Interpretable Shift Detection with NannyML](https://towardsdatascience.com/data-drift-explainability-interpretable-shift-detection-with-nannyml-83421319d05f/) by Marco Cerliani (Towards Data Science)
* **[Article]** [An End-to-End ML Model Monitoring Workflow with NannyML in Python](https://www.datacamp.com/tutorial/model-monitoring-with-nannyml-in-python) by Bex Tuychiyev (DataCamp)
* **[Article]** [Detecting Concept Drift: Impact on Machine Learning Performance](https://www.nannyml.com/blog/concept-drift) by Michal Oleszak (NannyML.Com)
* **[Article]** [Estimating Model Performance Without Labels](https://www.nannyml.com/blog/machine-learning-performance-estimation) by Jakub Białek (NannyML.Com)
* **[Article]** [Monitoring Workflow for Machine Learning Systems](https://www.nannyml.com/blog/machine-learning-monitoring-workflow) by Santiago Víquez (NannyML.Com)
* **[Article]** [Don’t Let Yourself Be Fooled by Data Drift](https://www.nannyml.com/blog/when-data-drift-does-not-affect-performance-machine-learning-models) by Santiago Víquez (NannyML.Com)
* **[Article]** [Understanding Data Drift: Impact on Machine Learning Model Performance](https://www.nannyml.com/blog/types-of-data-shift) by Jakub Białek (NannyML.Com)
* **[Article]** [NannyML’s Guide to Data Quality and Covariate Shift](https://www.nannyml.com/blog/guide-data-quality-covariate-shift) by Magdalena Kowalczuk (NannyML.Com)
* **[Article]** [From Reactive to Proactive: Shift your ML Monitoring Approach](https://www.nannyml.com/blog/proactive-ml-monitoring-workflow) by Qiamo (Luca) Zheng (NannyML.Com)
* **[Article]** [How to Detect Under-Performing Segments in ML Models](https://www.nannyml.com/blog/detecting-underpeforming--segments-in-ml-models) by Kavita Rana (NannyML.Com)
* **[Article]** [Building Custom Metrics for Predictive Maintenance](https://www.nannyml.com/blog/custom-metrics-predictive-maintenance) by Kavita Rana(NannyML.Com)
* **[Article]** [3 Custom Metrics for Your Forecasting Models](https://www.nannyml.com/blog/custom-metrics-for-demand-forecasting-models) by Kavita Rana (NannyML.Com)
* **[Article]** [There's Data Drift, But Does It Matter?](https://www.nannyml.com/blog/data-drift-does-it-matter) by Santiago Víquez (NannyML.Com)
* **[Article]** [Monitoring Custom Metrics without Ground Truth](https://www.nannyml.com/blog/monitoring-custom-metrics-without-ground-truth) by Kavita Rana (NannyML.Com)
* **[Article]** [Which Multivariate Drift Detection Method Is Right for You: Comparing DRE and DC](https://www.nannyml.com/blog/dre-vs-dc) by Miles Weberman (NannyML.Com)
* **[Article]** [Prevent Failure of Product Defect Detection Models: A Post-Deployment Guide](https://www.nannyml.com/blog/prevent-failure-of-product-defect-detection-models) by Kavita Rana (NannyML.Com)
* **[Article]** [Common Pitfalls in Monitoring Default Prediction Models and How to Fix Them](https://www.nannyml.com/blog/monitor-default-prediction-models) by Miles Weberman (NannyML.Com)
* **[Article]** [Why Relying on Training Data for ML Monitoring Can Trick You](https://www.nannyml.com/blog/choose-reference-dataset) by Kavita Rana (NannyML.Com)
* **[Article]** [Estimating Model Performance Without Labels](https://www.nannyml.com/blog/machine-learning-performance-estimation) by Jakub Białek (NannyML.Com)
* **[Article]** [Using Concept Drift as a Model Retraining Trigger](https://www.nannyml.com/blog/concept-drift-retraining-trigger) by Taliya Weinstein (NannyML.Com)
* **[Article]** [Retraining is Not All You Need](https://www.nannyml.com/blog/retraining-is-not-all-you-need) by Miles Weberman (NannyML.Com)
* **[Article]** [A Comprehensive Guide to Univariate Drift Detection Methods](https://www.nannyml.com/blog/comprehensive-guide-univariate-methods) by Kavita Rana (NannyML.Com)
* **[Article]** [Stress-free Monitoring of Predictive Maintenance Models](https://www.nannyml.com/blog/monitor-predictive-maintenance-models-stress-free) by Kavita Rana (NannyML.Com)
* **[Article]** [Effective ML Monitoring: A Hands-on Example](https://www.nannyml.com/blog/ml-monitoring-workflow-hands-on) by Miles Weberman (NannyML.Com)
* **[Article]** [Don’t Drift Away with Your Data: Monitoring Data Drift from Setup to Cloud](https://www.nannyml.com/blog/monitoring-data-drift) by Taliya Weinstein (NannyML.Com)
* **[Article]** [Comparing Multivariate Drift Detection Algorithms on Real-World Data](https://www.nannyml.com/blog/tutorial-multivariate-drift-comparison) by Kavita Rana (NannyML.Com)
* **[Article]** [Detect Data Drift Using Domain Classifier in Python](https://www.nannyml.com/blog/data-drift-domain-classifier) by Miles Weberman (NannyML.Com)
* **[Article]** [Guide: How to evaluate if NannyML is the right monitoring tool for you](https://www.nannyml.com/blog/evaluate-nannyml) by Santiago Víquez (NannyML.Com)
* **[Article]** [How To Monitor ML models with NannyML SageMaker Algorithms](https://www.nannyml.com/blog/how-to-monitor-ml-models-with-nannyml-sagemaker-algorithms) by Wiljan Cools (NannyML.Com)
* **[Article]** [Tutorial: Monitoring Missing and Unseen values with NannyML](https://www.nannyml.com/blog/monitoring-missing-values-tutorial) by Santiago Víquez (NannyML.Com)
* **[Article]** [Monitoring Machine Learning Models: A Fundamental Practice for Data Scientists and Machine Learning Engineers](https://medium.com/data-science/monitoring-machine-learning-models-a-tried-and-true-cure-for-a-data-scientists-insomnia-c45b0979a878) by Saurav Pawar (Medium)
* **[Article]** [Failure Is Not an Option: How to Prevent Your ML Model From Degradation](https://medium.com/nannyml/failure-is-not-an-option-how-to-prevent-your-ml-model-from-degradation-nannyml-4473898af34a) by Maciej Balawejder (Medium)
* **[Article]** [Managing Data Drift and Data Distribution Shifts in the MLOps Lifecycle for Machine Learning Models](https://abhishek-reddy.medium.com/detecting-and-managing-data-distribution-shifts-in-the-mlops-lifecycle-for-machine-learning-models-1ea33ce84c3c) by Abhishek Reddy (Medium)
* **[Article]** [“You Can’t Predict the Errors of Your Model”… Or Can You?](https://medium.com/data-science/you-cant-predict-the-errors-of-your-model-or-can-you-1a2e4a1f38a0) by Samuele Mazzanti (Medium)
* **[Article]** [Understanding Concept Drift: A Simple Guide](https://medium.com/data-science/understanding-concept-drift-a-simple-guide-b2cf4e09deae) by Vitor Cerqueira (Medium)
* **[Article]** [Detecting Covariate Shift: A Guide to the Multivariate Approach](https://medium.com/data-science/detecting-covariate-shift-a-guide-to-the-multivariate-approach-c099bd1891b9) by Michał Oleszak (Medium)
* **[Article]** [Data Drift vs. Concept Drift: Differences and How to Detect and Address Them](https://dataheroes.ai/blog/data-drift-vs-concept-drift/) by DataHeroes Team (DataHeroes.AI)
* **[Article]** [An Introduction to Machine Learning Engineering for Production /MLOps — Concept and Data Drifts](https://medium.com/data-science/an-introduction-to-machine-learning-engineering-for-production-part-1-2247bbca8a61) by Praatibh Surana (Medium)
* **[Article]** [Concept Drift and Model Decay in Machine Learning](https://medium.com/data-science/concept-drift-and-model-decay-in-machine-learning-a98a809ea8d4) by Ashok Chilakapati (Medium)
* **[Article]** [Data Drift: Types of Data Drift](https://medium.com/data-science/data-drift-part-1-types-of-data-drift-16b3eb175006) by Numal Jayawardena (Medium)
* **[Article]** [Monitoring Machine Learning models](https://medium.com/data-science/monitoring-machine-learning-models-62d5833c7ecc) by Jacques Verre (Medium)
* **[Article]** [Data drift: It Can Come At You From Anywhere](https://medium.com/data-science/data-drift-it-can-come-at-you-from-anywhere-b78eb186855) by Tirthajyoti Sarkar (Medium)
* **[Article]** [Drift in Machine Learning](https://medium.com/data-science/drift-in-machine-learning-e49df46803a) by Piotr (Peter) Mardziel (Medium)
* **[Article]** [Understanding Dataset Shift](https://medium.com/data-science/understanding-dataset-shift-f2a5a262a766) by Matthew Stewart (Medium)
* **[Article]** [Calculating Data Drift in Machine Learning using Python](https://medium.com/data-science/calculating-data-drift-in-machine-learning-53676ff5646b) by Vatsal (Medium)
* **[Article]** [91% of ML Models Degrade in Time](https://medium.com/data-science/91-of-ml-models-degrade-in-time-cfd467905615) by Santiago Víquez (Medium)
* **[Article]** [Model Drift in Machine Learning](https://medium.com/data-science/model-drift-in-machine-learning-8023e3d08217) by Kurtis Pykes (Medium)
* **[Article]** [Production Machine Learning Monitoring: Outliers, Drift, Explainers & Statistical Performance](https://medium.com/data-science/production-machine-learning-monitoring-outliers-drift-explainers-statistical-performance-d9b1d02ac158) by Alejandro Saucedo (Medium)
* **[Article]** [How to Detect Model Drift in MLOps Monitoring](https://medium.com/data-science/how-to-detect-model-drift-in-mlops-monitoring-7a039c22eaf9) by Amit Paka (Medium)
* **[Article]** [“My data drifted. What’s next?” How to handle ML model drift in production.](https://medium.com/data-science/my-data-drifted-whats-next-how-to-handle-ml-model-drift-in-production-78719ef007b1) by Elena Samuylova (Medium)
* **[Article]** [Machine Learning Model Drift](https://medium.com/data-science/machine-learning-model-drift-9cc43ad530d6) by Sophia Yang (Medium)
* **[Article]** [Estimating the Performance of an ML Model in the Absence of Ground Truth](https://medium.com/data-science/estimating-the-performance-of-an-ml-model-in-the-absence-of-ground-truth-cc87dbf6e57) by Eryk Lewinson (Medium)
* **[Article]** [Ensemble: Boosting, Bagging, and Stacking Machine Learning](https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/) by Jason Brownlee (MachineLearningMastery.Com)
* **[Article]** [Stacking Machine Learning: Everything You Need to Know](https://www.machinelearningpro.org/stacking-machine-learning/) by Ada Parker (MachineLearningPro.Org)
* **[Article]** [Ensemble Learning: Bagging, Boosting and Stacking](https://duchesnay.github.io/pystatsml/machine_learning/ensemble_learning.html) by Edouard Duchesnay, Tommy Lofstedt and Feki Younes (Duchesnay.GitHub.IO)
* **[Article]** [Stack Machine Learning Models: Get Better Results](https://developer.ibm.com/articles/stack-machine-learning-models-get-better-results/) by Casper Hansen (Developer.IBM.Com)
* **[Article]** [GradientBoosting vs AdaBoost vs XGBoost vs CatBoost vs LightGBM](https://www.geeksforgeeks.org/gradientboosting-vs-adaboost-vs-xgboost-vs-catboost-vs-lightgbm/) by Geeks for Geeks Team (GeeksForGeeks.Org)
* **[Article]** [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/) by Jason Brownlee (MachineLearningMastery.Com)
* **[Article]** [The Ultimate Guide to AdaBoost Algorithm | What is AdaBoost Algorithm?](https://www.mygreatlearning.com/blog/adaboost-algorithm/) by Ashish Kumar (MyGreatLearning.Com)
* **[Article]** [A Gentle Introduction to Ensemble Learning Algorithms](https://machinelearningmastery.com/tour-of-ensemble-learning-algorithms/) by Jason Brownlee (MachineLearningMastery.Com)
* **[Article]** [Ensemble Methods: Elegant Techniques to Produce Improved Machine Learning Results](https://www.toptal.com/machine-learning/ensemble-methods-machine-learning) by Necati Demir (Toptal.Com)
* **[Article]** [The Essential Guide to Ensemble Learning](https://www.v7labs.com/blog/ensemble-learning-guide) by Rohit Kundu (V7Labs.Com)
* **[Article]** [Develop an Intuition for How Ensemble Learning Works](https://machinelearningmastery.com/how-ensemble-learning-works/) by by Jason Brownlee (Machine Learning Mastery)
* **[Article]** [Mastering Ensemble Techniques in Machine Learning: Bagging, Boosting, Bayes Optimal Classifier, and Stacking](https://rahuljain788.medium.com/mastering-ensemble-techniques-in-machine-learning-bagging-boosting-bayes-optimal-classifier-and-c1dd8052f53f) by Rahul Jain (Medium)
* **[Article]** [Ensemble Learning: Bagging, Boosting, Stacking](https://ai.plainenglish.io/ml-tutorial-19-ensemble-learning-bagging-boosting-stacking-5a926db20ec5) by Ayşe Kübra Kuyucu (Medium)
* **[Article]** [Ensemble: Boosting, Bagging, and Stacking Machine Learning](https://medium.com/@senozanAleyna/ensemble-boosting-bagging-and-stacking-machine-learning-6a09c31thyroid_cancer778) by Aleyna Şenozan (Medium)
* **[Article]** [Boosting, Stacking, and Bagging for Ensemble Models for Time Series Analysis with Python](https://medium.com/@kylejones_47003/boosting-stacking-and-bagging-for-ensemble-models-for-time-series-analysis-with-python-d74ab9026782) by Kyle Jones (Medium)
* **[Article]** [Different types of Ensemble Techniques — Bagging, Boosting, Stacking, Voting, Blending](https://medium.com/@abhishekjainindore24/different-types-of-ensemble-techniques-bagging-boosting-stacking-voting-blending-b04355a03c93) by Abhishek Jain (Medium)
* **[Article]** [Mastering Ensemble Techniques in Machine Learning: Bagging, Boosting, Bayes Optimal Classifier, and Stacking](https://rahuljain788.medium.com/mastering-ensemble-techniques-in-machine-learning-bagging-boosting-bayes-optimal-classifier-and-c1dd8052f53f) by Rahul Jain (Medium)
* **[Article]** [Understanding Ensemble Methods: Bagging, Boosting, and Stacking](https://divyabhagat.medium.com/understanding-ensemble-methods-bagging-boosting-and-stacking-7683c493ac19) by Divya bhagat (Medium)
* **[Video Tutorial]** [Concept Drift Detection with NannyML | Webinar](https://www.youtube.com/watch?v=kBTty6JTW9Q) by NannyML (YouTube)
* **[Video Tutorial]** [Fooled by Data Drift: How to Monitor ML Without False Positives](https://www.youtube.com/watch?v=71vlPelFVs0) by NannyML (YouTube)
* **[Video Tutorial]** [Monitoring Custom Metrics Without Access to Targets](https://www.youtube.com/watch?v=tqPoaA0STHs) by NannyML (YouTube)
* **[Video Tutorial]** [Analyzing Your Model's Performance in Production](https://www.youtube.com/watch?v=qDufIIduw5M) by NannyML (YouTube)
* **[Video Tutorial]** [How to Monitor Predictive Maintenance Models | Webinar Replay](https://www.youtube.com/watch?v=VuEYjEKUkWA) by NannyML (YouTube)
* **[Video Tutorial]** [Machine Learning Monitoring Workflow [Webinar]](https://www.youtube.com/watch?v=aXyc9TSl_u8) by NannyML (YouTube)
* **[Video Tutorial]** [Monitoring Machine Learning Models on AWS | Webinar](https://www.youtube.com/watch?v=82NAJG8m5r0) by NannyML (YouTube)
* **[Video Tutorial]** [Root Cause Analysis for ML Model Failure](https://www.youtube.com/watch?v=f-15Ri8MFAM) by NannyML (YouTube)
* **[Video Tutorial]** [Quantifying the Impact of Data Drift on Machine Learning Model Performance | Webinar](https://www.youtube.com/watch?v=pZJQCxt0aus) by NannyML (YouTube)
* **[Video Tutorial]** [How to Detect Drift and Resolve Issues in Your Machine Learning Models?](https://www.youtube.com/watch?v=zkWDb2URdIQ) by NannyML (YouTube)
* **[Video Tutorial]** [Notebooks to Containers: Setting up Continuous (ML) Model Monitoring in Production](https://www.youtube.com/watch?v=00MJ-gvpRkI) by NannyML (YouTube)
* **[Video Tutorial]** [Performance Estimation using NannyML | Tutorial in Jupyter Notebook](https://www.youtube.com/watch?v=fo0ejdQFcT0) by NannyML (YouTube)
* **[Video Tutorial]** [What Is NannyML? Introducing Our Open Source Python Library](https://www.youtube.com/watch?v=HgZm2JdE5Fo) by NannyML (YouTube)
* **[Video Tutorial]** [How to Automatically Retrain Your Models with Concept Drift Detection?](https://www.youtube.com/watch?v=QoymE--4sPM) by NannyML (YouTube)
* **[Video Tutorial]** [How to Use NannyML? Two Modes of Running Our Library](https://www.youtube.com/watch?v=ATKJXo6lTls) by NannyML (YouTube)
* **[Video Tutorial]** [How to Integrate NannyML in Production? | Tutorial](https://www.youtube.com/watch?v=zdiM2ZFD__w) by NannyML (YouTube)
* **[Video Tutorial]** [Bringing Your Machine Learning Model to Production | Overview](https://www.youtube.com/watch?v=XH_4X6oo8hI) by NannyML (YouTube)
* **[Video Tutorial]** [Notebooks to Containers: Setting Up Continuous (ML) Model Monitoring in Production](https://www.youtube.com/watch?v=00MJ-gvpRkI) by NannyML (YouTube)
* **[Video Tutorial]** [ML Performance without Labels: Comparing Performance Estimation Methods (Webinar Replay)](https://www.youtube.com/watch?v=CKCSC9Srgnc) by NannyML (YouTube)
* **[Course]** [DataCamp Python Data Analyst Certificate](https://app.datacamp.com/learn/career-tracks/data-analyst-with-python) by DataCamp Team (DataCamp)
* **[Course]** [DataCamp Python Associate Data Scientist Certificate](https://app.datacamp.com/learn/career-tracks/associate-data-scientist-in-python) by DataCamp Team (DataCamp)
* **[Course]** [DataCamp Python Data Scientist Certificate](https://app.datacamp.com/learn/career-tracks/data-scientist-in-python) by DataCamp Team (DataCamp)
* **[Course]** [DataCamp Machine Learning Engineer Certificate](https://app.datacamp.com/learn/career-tracks/machine-learning-engineer) by DataCamp Team (DataCamp)
* **[Course]** [DataCamp Machine Learning Scientist Certificate](https://app.datacamp.com/learn/career-tracks/machine-learning-scientist-with-python) by DataCamp Team (DataCamp)
* **[Course]** [IBM Data Analyst Professional Certificate](https://www.coursera.org/professional-certificates/ibm-data-analyst) by IBM Team (Coursera)
* **[Course]** [IBM Data Science Professional Certificate](https://www.coursera.org/professional-certificates/ibm-data-science) by IBM Team (Coursera)
* **[Course]** [IBM Machine Learning Professional Certificate](https://www.coursera.org/professional-certificates/ibm-machine-learning) by IBM Team (Coursera)